In [17]:
import numpy as np
import random
import math

In [18]:
capacity = 20
max_movement = 7
max_value_iterations = 10000

lambda_demand1, lambda_demand2, lambda_return1, lambda_return2 = 3, 4, 3, 2
gamma = 0.9
actions = list(range(-max_movement, max_movement + 1))
theta = 0.01
" ".join([str(x) for x in actions])

'-7 -6 -5 -4 -3 -2 -1 0 1 2 3 4 5 6 7'

In [19]:
def is_feasible(s, a, car_capacity):
    if a > 0 and (s[1] + a > car_capacity):
        return False

    if a < 0 and (s[0] - a > car_capacity):
        return False

    return True

In [ ]:
def poisson(lam, n):
    for k in range(n):
        l_pois = [math.exp(-lam) * (lam ** k) / math.factorial(k)]

    l_pois.append(1-sum(l_pois)) # pois of 20, the remaining probability
    return l_pois

def initialization(car_capacity):
     v = 100 * np.random.uniform(size = [car_capacity + 1, car_capacity + 1])
     return v

In [ ]:
def new_value(v, s, a, gamma, car_capacity, l_prob_demand1, l_prob_demand2, l_prob_return1, l_prob_return2):
    if not is_feasible(s, a, car_capacity):
        raise Exception("N/A")

    p_next_states1, p_rented1 = np.zeros(car_capacity+1), np.zeros(car_capacity+1)

    s_0 = min(car_capacity,max(s[0] - a,0))
    s_1 = min(car_capacity,max(s[1] + a,0))


    for demand1 in range(car_capacity + 1):
        rented1 = int(min(s_0,demand1))
        not_rented1 = s_0 - rented1
        prob_demand1 = l_prob_demand1[demand1]
        for return1 in range(car_capacity + 1):
            next_state1 = int(min(not_rented1 + return1, car_capacity))
            p_joint = prob_demand1 * l_prob_return1[return1]
            p_next_states1[next_state1] += p_joint
            p_rented1[rented1] += p_joint

    p_next_states2, p_rented2 = np.zeros(car_capacity+1), np.zeros(car_capacity + 1)

    for demand2 in range(car_capacity + 1):
        rented2 = int(min(s_1,demand2))
        not_rented_2 = s_1 - rented2
        p_demand2 = l_prob_demand2[demand2]
        for return_2 in range(car_capacity + 1):
            next_state2 = int(min(not_rented_2 + return_2, car_capacity))
            p_joint = p_demand2 * l_prob_return2[return_2]
            p_next_states2[next_state2] += p_joint
            p_rented2[rented2] += p_joint

    result = -2 * abs(a)
    for state_1 in range(car_capacity + 1):
        for state_2 in range(car_capacity + 1):
            result += p_next_states1[state_1]*p_next_states2[state_2] * gamma * v[state_1,state_2]

    for rented1 in range(car_capacity + 1):
        result += p_rented1[rented1]*rented1 * 10

    for rented2 in range(car_capacity + 1):
        result += p_rented2[rented2]*rented2 * 10

    return result